### Some simple tests for the different types of reflectivity in cv_synth

##### Updated for EI, January 30th, 2018

In [1]:
import os
import numpy as np

# small4 Model in Pem2Seis

In [2]:
def calculated_EEI(Vp0, Vs0, rho0, K, chi, Vp, Vs, rho):
    
    """Using Whitcombe(2002)
    Vp0, Vs0, rho0 are the reference constants, chi is the angle in degrees
    output is EEI"""
    
    chi_rad = chi*(np.pi/180)
    p = np.cos(chi_rad) + np.sin(chi_rad)
    q = -8*K * np.sin(chi_rad)
    r = np.cos(chi_rad) - (4*K*np.sin(chi_rad))
    EEI = Vp0 * rho0 * ((Vp/Vp0)**p) * ((Vs/Vs0)**q) * ((rho/rho0)**r)
    return EEI   
    

In [3]:
def reflectivity(Imp_above, Imp_below):
    
    """Take Imp_above and Imp_below and return the reflectivity"""
    
    Reflectivity = (Imp_below - Imp_above) / (Imp_below + Imp_above)
    print("\n\nBelow Imp =", Imp_below,": Above Imp =", Imp_above,": Reflectivity =", Reflectivity)
    return Reflectivity

In [4]:
def convolution(reflectivity):
    
    """"Reads in a DGI temp.wvl file
    Output at any point in the trace is the convolution of that wavelet
    with the reflectivity at that point. In cv_synth this is true for 
    both the Band-limted impendace and the reflectivity volume. Both are
    just a convolution of the EEI or reflectivity series with the wavelet"""
    
    os.chdir('d:\\users\\graham\\Dropbox\\Python\\Reflectivity_and_Wavelets\\reflectivity_testing')
    wvlines = open('temp.wvl','r').readlines()[4:]
    wavelet = np.array([float(x.strip()) for x in wvlines])
    amplitude = 0    
    for amp in wavelet:
        amplitude += amp * reflectivity
    return amplitude    

In [5]:
def shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta1, terms=False):
    
    """
    Taken from BRUGES
    Compute Shuey approximation with 3 terms.
    http://subsurfwiki.org/wiki/Shuey_equation
    :param vp1: The p-wave velocity of the upper medium.
    :param vs1: The s-wave velocity of the upper medium.
    :param rho1: The density of the upper medium.
    :param vp2: The p-wave velocity of the lower medium.
    :param vs2: The s-wave velocity of the lower medium.
    :param rho2: The density of the lower medium.
    :param theta1: An array of incident angles to use for reflectivity
                   calculation [degrees].
    :returns: a vector of len(theta1) containing the reflectivity
             value corresponding to each angle.
    """
    
    theta1 = np.radians(theta1)

    # Compute some parameters
    drho = rho2-rho1
    dvp = vp2-vp1
    dvs = vs2-vs1
    rho = (rho1+rho2)/2.0
    vp = (vp1+vp2)/2.0
    vs = (vs1+vs2)/2.0

    # Compute three-term reflectivity
    r0 = 0.5 * (dvp/vp + drho/rho)
    g = 0.5 * dvp/vp - 2 * (vs**2/vp**2) * (drho/rho + 2 * dvs/vs)
    f = 0.5 * dvp/vp

    term1 = r0
    term2 = g * np.sin(theta1)**2
    term3 = f * (np.tan(theta1)**2 - np.sin(theta1)**2)

    if terms:
        return term1, term2, term3
    else:
        return (term1 + term2 + term3)

In [6]:
def akirichards(vp1, vs1, rho1, vp2, vs2, rho2, theta1, terms=False):
    
    """
    This is the formulation from Avseth et al.,
    Quantitative seismic interpretation,
    Cambridge University Press, 2006. Adapted for a 4-term formula.
    See http://subsurfwiki.org/wiki/Aki-Richards_equation
    :param vp1: The p-wave velocity of the upper medium.
    :param vs1: The s-wave velocity of the upper medium.
    :param rho1: The density of the upper medium.
    :param vp2: The p-wave velocity of the lower medium.
    :param vs2: The s-wave velocity of the lower medium.
    :param rho2: The density of the lower medium.
    :param theta1: An array of incident angles to use for reflectivity
                   calculation [degrees].
    :returns: a vector of len(theta1) containing the reflectivity
             value corresponding to each angle
    """

    # We are not using this for anything, but will
    # critical_angle = arcsin(vp1/vp2)

    # Do we need to ensure that we get floats out before
    # computing sines?
    if np.ndim(vp1) == 0:
        vp1 = float(vp1)
    else:
        vp1 = np.array(vp1).astype(float)

    theta1 = np.radians(theta1)
    theta2 = np.arcsin(vp2/vp1*np.sin(theta1))

    # Compute the various parameters
    drho = rho2-rho1
    dvp = vp2-vp1
    dvs = vs2-vs1
    meantheta = (theta1+theta2) / 2.0
    rho = (rho1+rho2) / 2.0
    vp = (vp1+vp2) / 2.0
    vs = (vs1+vs2) / 2.0

    # Compute the coefficients
    w = 0.5 * drho/rho
    x = 2 * (vs/vp1)**2 * drho/rho
    y = 0.5 * (dvp/vp)
    z = 4 * (vs/vp1)**2 * (dvs/vs)

    # Compute the terms
    term1 = w
    term2 = -1 * x * np.sin(theta1)**2
    term3 = y / np.cos(meantheta)**2
    term4 = -1 * z * np.sin(theta1)**2

    if terms:
        return term1, term2, term3, term4
    else:
        return (term1 + term2 + term3 + term4)

In [7]:
def elastic_impedance(vp, vs, rho, theta1,
                      k, Vp0, Vs0, rho0, 
                      normalize=True,
                      use_sin=False,
                      rho_term=False):
    
    constants = [Vp0, Vs0, rho0]
    
    """
    Returns the elastic impedance as defined by Connolly, 1999;
    we are using the formulation reported in Whitcombe et al. (2001).
    Inputs should be shape m x 1, angles should be n x 1. The
    result will be m x n.
    :param vp1: The P-wave velocity scalar or 1D array.
    :param vs1: The S-wave velocity scalar or 1D array.
    :param rho1: The bulk density scalar or 1D array.
    :param theta1: Incident angle(s), scalar or array [degrees].
    :param k: A constant, see Connolly (1999). Default is 'auto', which
        computes it from Vp and Vs
    :param normalize: if True, returns the normalized form of Whitcombe
        et. al (2001).
    :param constants: A sequence of 3 constants to use for normalization. If
        you don't provide this, then normalization just uses the means of the
        inputs. If these are scalars, the result will be the acoustic impedance
        (see Whitcombe, 2001).
    :param use_sin: Use sin^2 for the first term, instead of tan^2 (see
        Connolly 1999).
    :param rho_term: Alternative form, with Vp factored out; use in place of
        density in generating synthetics in other software (see Connolly 1999).
    """
    theta1 = np.array(np.radians(theta1)).reshape((-1, 1))
    if (np.nan_to_num(theta1) > np.pi/2.).any():
        raise ValueError("Incidence angle theta1 must be less than 90 deg.")

    alpha = np.array(vp, dtype=float)
    beta = np.array(vs, dtype=float)
    rho = np.array(rho, dtype=float)

    if use_sin:
        op = np.sin
    else:
        op = np.tan

    if k.lower() == 'auto':
        k = np.mean(beta**2.0 / alpha**2.0)
    # Otherwise, just use the k we were given.
    else:
        k=float(k)
    
    a = 1 + op(theta1)**2.0
    b = -8 * k * np.sin(theta1)**2.0
    c = 1 - 4 * k * np.sin(theta1)**2.0

    ei = alpha**a * beta**b * rho**c

    if normalize:
        if constants is None:
            # Use the means; this will return acoustic impedance for scalars.
            alpha_0 = np.nanmean(vp)
            beta_0 = np.nanmean(vs)
            rho_0 = np.nanmean(rho)
        else:
            try:
                alpha_0, beta_0, rho_0 = constants
            except ValueError as e:
                raise ValueError("You must provide a sequence of 3 constants.")
        ei *= alpha_0**(1 - a) * beta_0**(-b) * rho_0**(1 - c)

    if rho_term:
        ei /= alpha

    if ei.size == 1:
        return np.asscalar(ei)
    else:
        return np.squeeze(ei.T)

In [ ]:
def PS_impedance(vp, vs, rho, theta1,
                      k, Vp0, Vs0, rho0, 
                      normalize=True,
                      use_sin=False,
                      rho_term=False):
    
    '''
    This returns PSEI (P- to S- wave converted elastic impedance)
        c, d, are the the exponents of the density and velocity 
        factors respectively in Gonzalez Eqn 2.15
    '''
    assert K < 1
    root = sqrt(1./K**2 - sin(th)**2)
    c = K*sin(th)*(2*sin(th)**2 - 1/K**2 - 2*cos(th)*root)/root
    
    d = 4*K*sin(th)*(sin(th)**2-cos(th)*root)/root
    
    psei = xxxxx
    
    psei_norm = 
    
    return
    

In [14]:
##########################
#
# Set-up some variables for the experiment with small4.c3grd
#
##########################

Vp0 = 5600
Vs0 = 1650
rho0 = 1.65
K = 0.25
chi = 0

Vp_above = 4500
Vs_above = 1500
rho_above = 1.5

Vp_cell = 6000
Vs_cell = 1500
rho_cell = 1.5

!rm -rf small*
!cp -r ./backup/small* .

print("\nCell 2 1 10: timestep Oct 11 2012: EEI of cell = ", calculated_EEI(Vp0, Vs0, rho0, K, chi, Vp_cell, Vs_cell, rho_cell))
print("\nCell 2 1 10: timestep Oct 11 2012: EEI Reflectivity  = ", reflectivity(calculated_EEI(Vp0, Vs0, rho0, K, chi, Vp_above, Vs_above, rho_above), calculated_EEI(Vp0, Vs0, rho0, K, chi, Vp_cell, Vs_cell, rho_cell))) 
print("\nCell 2 1 10: timestep Oct 11 2012: Shuey Reflectivity  = ", shuey(Vp_above, Vs_above, rho_above, Vp_cell, Vs_cell, rho_cell, chi)) 
print("\nCell 2 1 10: timestep Oct 11 2012: A-K Reflectivity  = ", akirichards(Vp_above, Vs_above, rho_above, Vp_cell, Vs_cell, rho_cell, chi)) 

print("\nCell 2 1 10: timestep Oct 11 2012: Elastic Impedance  = ", elastic_impedance(Vp_cell, Vs_cell, rho_cell, chi, str(K), Vp0, Vs0, rho0))
print("\nCell 2 1 10: timestep Oct 11 2012: EI Reflectivity  = ", reflectivity(elastic_impedance(Vp_above, Vs_above, rho_above, chi, str(K), Vp0, Vs0, rho0), elastic_impedance(Vp_cell, Vs_cell, rho_cell, chi, str(K), Vp0, Vs0, rho0))) 




The system cannot find the path specified.



Cell 2 1 10: timestep Oct 11 2012: EEI of cell =  9000.0


Below Imp = 9000.0 : Above Imp = 6750.000000000001 : Reflectivity = 0.1428571428571428

Cell 2 1 10: timestep Oct 11 2012: EEI Reflectivity  =  0.1428571428571428

Cell 2 1 10: timestep Oct 11 2012: Shuey Reflectivity  =  0.14285714285714285

Cell 2 1 10: timestep Oct 11 2012: A-K Reflectivity  =  0.14285714285714285

Cell 2 1 10: timestep Oct 11 2012: Elastic Impedance  =  9000.0


Below Imp = 9000.0 : Above Imp = 6750.0 : Reflectivity = 0.14285714285714285

Cell 2 1 10: timestep Oct 11 2012: EI Reflectivity  =  0.14285714285714285


The system cannot find the path specified.


In [15]:
# Generate Pem2Seis parameters file
! rm pem2seis_simple.txt
fml = open('pem2seis_simple.txt','w')

fml.write('VEL_Pname=VEL_P\n')
fml.write('VEL_P_0='+ str(Vp0) +'\n')
fml.write('VEL_P_ABOVE='+ str(Vp_above) +'\n')    
fml.write('VEL_P_NULL='+ str(Vp_above) +'\n')    
fml.write('VEL_P_BELOW='+ str(Vp_above) +'\n')        
fml.write('VEL_Sname=VEL_S\n')
fml.write('VEL_S_0=' + str(Vs0) + '\n')
fml.write('VEL_S_ABOVE=' + str(Vs_above) + '\n')
fml.write('VEL_S_NULL=' + str(Vs_above) + '\n')
fml.write('VEL_S_BELOW=' + str(Vs_above) + '\n')    
fml.write('BULK_DENname=BULK_DEN\n')    
fml.write('BULK_DEN_0=' + str(rho0) + '\n')    
fml.write('BULK_DEN_ABOVE=' + str(rho_above) + '\n')    
fml.write('BULK_DEN_NULL=' + str(rho_above) + '\n')    
fml.write('BULK_DEN_BELOW=' + str(rho_above) + '\n')    
fml.write('CHI=' + str(chi) + '\n')    
fml.write('K=' + str(K) + '\n')    
fml.write('DATE=2012-10-11\n')    
       
fml.close()

The system cannot find the path specified.


In [16]:
# Now try the same thing in cv_synth
!rm -rf gb_ref*
!rm -rf gb_*
!rm -rf rf_series*
!rm -rf *wvl

!cv_waveletgen -N 1 -o temp.wvl -w ricker -1 9999 -s 0.05 -n 513

# with defaults
# !cv_synth -f pem2seis_simple.txt -t "template.3grd" -w temp.wvl -e graham -o "gb_eei_reflectivity" -r "small4.c3grd"

# with Shuey
#!cv_synth -f pem2seis_simple.txt -t "template.3grd" -w temp.wvl -sh -o "gb_shuey" -r "small4.c3grd"

# with Aki-Richards
#!cv_synth -f pem2seis_simple.txt -t "template.3grd" -w temp.wvl -ak -o "gb_aki" -r "small4.c3grd"

# with Connellys EI
!cv_synth -f pem2seis_simple.txt -t "template.3grd" -w temp.wvl -ei -e graham -o "gb_EI" -r "small4.c3grd"


!coviz small4.c3grd "gb_EI_rf_series_angle_15_2012-10-11.3grd"

#!coviz small4.c3grd "gb_aki_ref_angle_30_2012-10-11.3grd" "gb_aki_rf_series_angle_30_2012-10-11.3grd" \
#                    "gb_eei_reflectivity_ref_chi_30_2012-10-11.3grd" "gb_eei_reflectivity_rf_series_chi_30_2012-10-11.3grd" \
#                    "gb_shuey_ref_angle_30_2012-10-11.3grd" "gb_shuey_rf_series_angle_30_2012-10-11.3grd" \
#                    "gb_ei_ref_angle_30_2012-10-11.3grd" "gb_ei_rf_series_angle_30_2012-10-11.3grd"
        


The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.


arg 0: coviz
arg 1: small4.c3grd
arg 2: gb_EI_rf_series_angle_15_2012-10-11.3grd


The system cannot find the path specified.
